In [178]:
import io
import numpy as np
import pandas as pd
import requests
from pprint import pprint
from tqdm import tqdm

In [179]:
def is_resp_content_type_json(response) -> bool:
    return 'application/json' in response.headers['Content-Type']

In [180]:
def hubeau_get_stations_ades(numéro_département: int) -> pd.DataFrame:
    """Récupérer la liste (fichier CSV) des points d'eau d'un département et la retourner sous forme d'un DataFrame.
    Exemple de requête :
    https://hubeau.eaufrance.fr/api/v1/niveaux_nappes/stations.csv?code_departement=66&size=200
    """
    url = 'https://hubeau.eaufrance.fr/api/v1/niveaux_nappes/stations.csv'
    grand_nombre_magique = 9999
    query_params = {'code_departement': numéro_département, 'size': grand_nombre_magique}
    response = requests.get(url, params=query_params)
    df_liste_stations = pd.read_csv(io.BytesIO(response.content), sep=';')
    return df_liste_stations

In [181]:
def get_chroniques_station(infos_station: dict) -> pd.DataFrame:
    """Récupérer les données d'intérêt sur le code BSS d'une station.
    Exemple de requête :
    https://hubeau.eaufrance.fr/api/v1/niveaux_nappes/chroniques.csv?code_bss=10908X0136/DUCUP2&size=20000
    """
    url = 'https://hubeau.eaufrance.fr/api/v1/niveaux_nappes/chroniques.csv'
    grand_nombre_magique = 20000
    query_params = {
        'code_bss': infos_station.get('code_bss'), 'size': grand_nombre_magique}
    response = requests.get(url, params=query_params)
    if response.headers.get('Content-Length') == 0 \
            or 'Content-Encoding' not in response.headers:
        return None
    df = pd.read_csv(io.BytesIO(response.content), sep=';')
    return df

In [182]:
def hubeau_get_chroniques_des_stations(df_liste_stations: pd.DataFrame) -> pd.DataFrame:
    """Récupérer, pour chaque station de la liste, les données d'intérêt.
    """
    infos_stations_liste = df_liste_stations.to_dict(orient='records')
    infos_station0, le_reste_à_parcourir = infos_stations_liste[0], infos_stations_liste[1:]
    df_chroniques_station0 = get_chroniques_station(infos_station0)
    if df_chroniques_station0 is None:
        return None
    df_full = pd.merge(left=pd.DataFrame([infos_station0]),
                                               right=df_chroniques_station0,
                                               on=['code_bss'], how='outer')
    with tqdm(total=len(le_reste_à_parcourir), ncols=100) as progr_bar:
        for infos_station in le_reste_à_parcourir:
            progr_bar.set_description(f"Station {infos_station.get('code_bss')}")
            df_chroniques_station = get_chroniques_station(infos_station)
            progr_bar.update()
            if df_chroniques_station is None:
                continue
            df_tmp = pd.merge(left=pd.DataFrame([infos_station0]),
                            right=df_chroniques_station0,
                            on=['code_bss'], how='outer')
            df_full = pd.concat([df_full, df_tmp])
    return df_full

In [183]:
numéro_département = 66

df_liste_stations = hubeau_get_stations_ades(numéro_département)

df_infos_stations_et_chroniques = hubeau_get_chroniques_des_stations(df_liste_stations)

nom_fichier_destination = f'hubeau_chroniques_des_stations_ades_dans_{numéro_département}.csv'

df_infos_stations_et_chroniques.to_csv(nom_fichier_destination, sep=';', index=False)

Station 10915X0395/PZ: 100%|████████████████████████████████████████| 85/85 [02:12<00:00,  1.55s/it]


In [184]:
if df_infos_stations_et_chroniques is None:
    df_infos_stations_et_chroniques = pd.read_csv(nom_fichier_destination, sep=';')
df_infos_stations_et_chroniques

,code_bss,urn_bss_x,date_debut_mesure,date_fin_mesure,code_commune_insee,nom_commune,x,y,codes_bdlisa,urns_bdlisa,...,mode_obtention,statut,qualification,code_continuite,nom_continuite,code_producteur,nom_producteur,code_nature_mesure,nom_nature_mesure,profondeur_nappe
0,10906X0039/C2-1,http://services.ades.eaufrance.fr/pointeau/109...,2000-04-14,2023-08-20,66108,Millas,2.685378,42.685696,718BP02,http://reseau.eaufrance.fr/geotraitements/bdli...,...,Valeur mesurée,Donnée contrôlée niveau 1,Correcte,2,Point lié au point précédent,231,Bureau de Recherches Géologiques et Minières (...,NaN,NaN,4.85
1,10906X0039/C2-1,http://services.ades.eaufrance.fr/pointeau/109...,2000-04-14,2023-08-20,66108,Millas,2.685378,42.685696,718BP02,http://reseau.eaufrance.fr/geotraitements/bdli...,...,Valeur mesurée,Donnée contrôlée niveau 1,Correcte,2,Point lié au point précédent,231,Bureau de Recherches Géologiques et Minières (...,NaN,NaN,4.87
2,10906X0039/C2-1,http://services.ades.eaufrance.fr/pointeau/109...,2000-04-14,2023-08-20,66108,Millas,2.685378,42.685696,718BP02,http://reseau.eaufrance.fr/geotraitements/bdli...,...,Valeur mesurée,Donnée contrôlée niveau 1,Correcte,2,Point lié au point précédent,231,Bureau de Recherches Géologiques et Minières (...,NaN,NaN,4.88
3,10906X0039/C2-1,http://services.ades.eaufrance.fr/pointeau/109...,2000-04-14,2023-08-20,66108,Millas,2.685378,42.685696,718BP02,http://reseau.eaufrance.fr/geotraitements/bdli...,...,Valeur mesurée,Donnée contrôlée niveau 1,Correcte,2,Point lié au point précédent,231,Bureau de Recherches Géologiques et Minières (...,NaN,NaN,4.87
4,10906X0039/C2-1,http://services.ades.eaufrance.fr/pointeau/109...,2000-04-14,2023-08-20,66108,Millas,2.685378,42.685696,718BP02,http://reseau.eaufrance.fr/geotraitements/bdli...,...,Valeur mesurée,Donnée contrôlée niveau 1,Correcte,2,Point lié au point précédent,231,Bureau de Recherches Géologiques et Minières (...,NaN,NaN,4.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7847,10906X0039/C2-1,http://services.ades.eaufrance.fr/pointeau/109...,2000-04-14,2023-08-20,66108,Millas,2.685378,42.685696,718BP02,http://reseau.eaufrance.fr/geotraitements/bdli...,...,Valeur mesurée,Donnée brute,Non qualifié,2,Point lié au point précédent,20001584000016,Syndicat Mixte pour la Protection et la Gestio...,N,Naturel,4.34
7848,10906X0039/C2-1,http://services.ades.eaufrance.fr/pointeau/109...,2000-04-14,2023-08-20,66108,Millas,2.685378,42.685696,718BP02,http://reseau.eaufrance.fr/geotraitements/bdli...,...,Valeur mesurée,Donnée brute,Non qualifié,2,Point lié au point précédent,20001584000016,Syndicat Mixte pour la Protection et la Gestio...,N,Naturel,4.33
7849,10906X0039/C2-1,http://services.ades.eaufrance.fr/pointeau/109...,2000-04-14,2023-08-20,66108,Millas,2.685378,42.685696,718BP02,http://reseau.eaufrance.fr/geotraitements/bdli...,...,Valeur mesurée,Donnée brute,Non qualifié,2,Point lié au point précédent,20001584000016,Syndicat Mixte pour la Protection et la Gestio...,N,Naturel,4.32
7850,10906X0039/C2-1,http://services.ades.eaufrance.fr/pointeau/109...,2000-04-14,2023-08-20,66108,Millas,2.685378,42.685696,718BP02,http://reseau.eaufrance.fr/geotraitements/bdli...,...,Valeur mesurée,Donnée brute,Non qualifié,2,Point lié au point précédent,20001584000016,Syndicat Mixte pour la Protection et la Gestio...,N,Naturel,4.32
